# Load dataset

In [ ]:
import json

# Once we have the json file, we can process it
with open('millionaire_questions.json', 'r') as file:
    data = json.load(file)

all_questions = [x for qs in data.values() for x in qs]
print('total questions:', len(all_questions))
print('word count:', len(' '.join(all_questions).split()))

total questions: 25865
word count: 430258


In [ ]:
import pandas as pd

df = {'Contestant':[], 'Question':[]}
for contestant,questions in data.items():
    for question in questions:
        if '?' in question:
            qsplit = question.split('?')
            if len(qsplit) == 1:
                df['Contestant'].append(contestant)
                df['Question'].append(question)
            else:
                cleaned_q = '?'.join(qsplit[:-1])+'?'
                df['Contestant'].append(contestant)
                df['Question'].append(cleaned_q)

pd.DataFrame(df)

,Contestant,Question
0,Aaron Gold,"According to Good Housekeeping, most homeowner..."
1,Aaron Gold,"""There is a Sucker Born Ev'ry Minute"" and ""Joi..."
2,Aaron Gold,"Until it adopted the euro in 2002, fittingly, ..."
3,Aaron Gold,Which of these is the correct spelling of what...
4,Aaron Hauck,"In 2006, the WHO reported that thousands of pe..."
...,...,...
24790,Zeke Spector,"As Crayola's website explains, ""while artists ..."
24791,Zeke Spector,"Born Amethyst Amelia Kelly, what recording art..."
24792,Zeke Spector,Which one of these things is NOT true of the p...
24793,Zeke Spector,Since in 1992 it was ruled that anyone in the ...


# Berkeley nerual parser

In [ ]:
!pip install -q benepar
!python -m spacy download en_core_web_md

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 17.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.loa

In [ ]:
import benepar
benepar.download('benepar_en3_large')

[nltk_data] Downloading package benepar_en3_large to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3_large.zip.


True

In [ ]:
# Demo example (https://pypi.org/project/benepar/)
import spacy
nlp = spacy.load('en_core_web_md')
if spacy.__version__.startswith('2'):
    nlp.add_pipe(benepar.BeneparComponent("benepar_en3_large"))
else:
    nlp.add_pipe("benepar", config={"model": "benepar_en3_large"})

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# Check whether GPU is working
print(next(nlp.model.parameters()).device)

In [ ]:
doc = nlp("The time for action is now. It's never too late to do something.")
sent = list(doc.sents)[0]
print(sent._.parse_string)
print(sent._.labels)
list(sent._.children)[0]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/distributions/distribution.py:56: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


(S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
('S',)


The time for action

In [ ]:
test_q = df['Question'][0]

test_data = nlp(test_q)

sent = list(test_data.sents)[0]
print(sent._.parse_string)
print(sent._.labels)
list(sent._.children)[0]

(S (PP (VBG According) (PP (IN to) (NP (NNP Good) (NNP Housekeeping)))) (, ,) (NP (JJS most) (NNS homeowners)) (VP (VBP have) (S (VP (TO to) (VP (VB do) (NP (NP (DT all) (CC but) (WDT which)) (PP (IN of) (NP (DT these) (NNS chores)))) (PP (RB once) (NP (DT a) (NN year))))))) (. ?))
('S',)


According to Good Housekeeping

In [ ]:
# ChatGPT helping to clean text
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Normalize all whitespace to single space
    text = text.replace('\xa0', '')  # Replace non-breaking space
    text = text.strip()               # Remove leading/trailing spaces
    return text

raw_questions = [clean_text(question) for question in df['Question']]

In [ ]:
from tqdm import tqdm

df['Parse'] = []

for question in tqdm(raw_questions):
    nlp_data = nlp(question)
    sent = list(nlp_data.sents)[0]
    tree = sent._.parse_string
    df['Parse'].append(tree)

100%|██████████| 24795/24795 [39:43<00:00, 10.40it/s]


In [ ]:
parsed_questions = pd.DataFrame(df)
parsed_questions.to_csv('parsed_questions_benepar.csv', index=False)
parsed_questions

,Contestant,Question,Parse
0,Aaron Gold,"According to Good Housekeeping, most homeowner...",(S (PP (VBG According) (PP (IN to) (NP (NNP Go...
1,Aaron Gold,"""There is a Sucker Born Ev'ry Minute"" and ""Joi...","(S (NP (`` "") (S (NP (EX There)) (VP (VBZ is) ..."
2,Aaron Gold,"Until it adopted the euro in 2002, fittingly, ...",(SBARQ (SBAR (IN Until) (S (NP (PRP it)) (VP (...
3,Aaron Gold,Which of these is the correct spelling of what...,(SBARQ (WHNP (WHNP (WDT Which)) (PP (IN of) (N...
4,Aaron Hauck,"In 2006, the WHO reported that thousands of pe...","(S (PP (IN In) (NP (CD 2006))) (, ,) (NP (DT t..."
...,...,...,...
24790,Zeke Spector,"As Crayola's website explains, ""while artists ...",(S (SBAR (IN As) (S (NP (NP (NNP Crayola) (POS...
24791,Zeke Spector,"Born Amethyst Amelia Kelly, what recording art...",(SBARQ (S (VP (VBN Born) (NP (NNP Amethyst) (N...
24792,Zeke Spector,Which one of these things is NOT true of the p...,(SBARQ (WHNP (NP (WDT Which) (CD one)) (PP (IN...
24793,Zeke Spector,Since in 1992 it was ruled that anyone in the ...,(S (SBAR (IN Since) (S (PP (IN in) (NP (CD 199...


In [ ]:
question = df['Question'][23041]
if question.startswith('\xa0'):
  question = question[1:]
question

'What branch of the U.S. military promotes itself as "The few.  The proud."?'

# Stanza parse (Abandoned)

Originally I tried using Stanford's CoreNLP 'stanza' parser, but it turned out not to be very accurate...

In [ ]:
!pip install -q stanza
import stanza
stanza.download('en', verbose=False) # download English model

import torch
if torch.cuda.is_available():
    device, use_gpu = 'cuda', True
else:
    device, use_gpu = 'cpu', False

nlp = stanza.Pipeline('en', use_gpu=use_gpu, device=device) # initialize English neural pipeline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 37.3 MB/s eta 0:00:00


In [ ]:
test_qs = ' '.join(df['Question'])

test_data = nlp(test_qs)

test_trees = [sentence.constituency for sentence in test_data.sentences]
len(test_trees)

24981

In [ ]:
len(df['Question'])

24795

In [ ]:
from tqdm import tqdm

df['Parse'] = []

for question in tqdm(df['Question']):
    nlp_data = nlp(question)
    tree = nlp_data.sentences[0].constituency
    df['Parse'].append(tree)

100%|██████████| 24795/24795 [1:34:02<00:00,  4.39it/s]


In [ ]:
pd.DataFrame(df).to_csv('parsed_questions_stanza.csv', index=False)